In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle datasets download -d jessicali9530/celeba-dataset

 99% 1.20G/1.21G [00:20<00:00, 87.9MB/s]
100% 1.21G/1.21G [00:20<00:00, 62.3MB/s]


In [8]:
!unzip \*.zip

Archive:  celeba-dataset.zip
  inflating: list_attr_celeba.csv    
  inflating: img_align_celeba.zip    
  inflating: list_landmarks_align_celeba.csv  
  inflating: list_bbox_celeba.csv    
  inflating: list_eval_partition.csv  


In [0]:
import pandas as pd
d = pd.read_csv('list_attr_celeba.csv')
d.head()

In [0]:
!unzip img_align_celeba

In [0]:
!mkdir sample

In [11]:
import os
from PIL import Image
from tqdm import tqdm

def process(folder, dest_folder, target_shape=None, from_ending=None, to_ending=None,grayscale=False):
      if os.path.isfile(folder):
            try:
                  if from_ending is None or folder.endswith(from_ending):
                        img = Image.open(folder)
                        if target_shape is not None:
                              img = img.resize(target_shape)
                        if grayscale:
                              img = img.convert('L')
                        if to_ending is not None:
                              ending = to_ending
                        else:
                              ending = folder[folder.rfind('.'):]
                        num_processed = len(os.listdir(dest_folder))
                        img_name = str(num_processed) + ending
                        img_name = os.path.join(dest_folder, img_name)
                        img.save(img_name)
            except Exception as e:
                  print('Failed for file %s: %s' % (folder, str(e)))
      elif os.path.isdir(folder):
            files = os.listdir(folder)
            files=files[:40000]
            for file in tqdm(files):
                  process(os.path.join(folder,file),
                          dest_folder,
                          target_shape=target_shape,
                          from_ending=from_ending,
                          to_ending=to_ending,
                          grayscale=grayscale)
      else:
            print('ERROR: %s does not exist!' % folder)

process(r'img_align_celeba',
        r'sample',target_shape=(64,64),
        from_ending='.jpg',
        to_ending='.png',
        grayscale=True)

100%|██████████| 40000/40000 [07:45<00:00, 86.01it/s]


In [12]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

img_dir = r'sample'
files = [x for x in os.listdir(img_dir) if x[-4:] == '.png']
dataset = np.empty((len(files),64,64,1))
for i,file in tqdm(enumerate(files)):
      img = Image.open(os.path.join(img_dir, file))
      img = np.array(img)
      if len(img.shape) > 2:
            img = img[:,:,0]
      dataset[i] = np.expand_dims(img,axis=2)
np.save('dataset.npy', dataset)

40000it [00:08, 4659.00it/s]


In [13]:
ls

celeba-dataset.zip    kaggle.json              list_landmarks_align_celeba.csv
dataset.npy           list_attr_celeba.csv     sample/
img_align_celeba/     list_bbox_celeba.csv     sample_data/
img_align_celeba.zip  list_eval_partition.csv


In [0]:
import numpy as np

def rescale(data, min_num, max_num, data_min=None, data_max=None):
      if data_min is None:
            data_min = np.min(data)
      if data_max is None:
            data_max = np.max(data)
      data_range = data_max - data_min
      data = ((data - data_min) / data_range) * (max_num - min_num) + min_num
      return data

def write_log(callback, names, logs, epoch):
      import tensorflow as tf
      """Write stats to TensorBoard"""
      for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, epoch)
        callback.writer.flush()

In [15]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, GaussianNoise, LeakyReLU, Dropout
from keras.layers import BatchNormalization, Flatten, Dense, Activation
from keras.layers import Conv2DTranspose, Input, Reshape
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import os
from datetime import datetime
import math
import numpy as np
from PIL import Image
from tqdm import tqdm

np.random.seed(26)

image_shape = (64,64,1)
noise_shape = (100,)

def build_discriminator():
      discriminator = Sequential()
      discriminator.add(GaussianNoise(0.01, input_shape=image_shape))
      discriminator.add(Conv2D(64, kernel_size=3, strides=1, padding='same'))
      discriminator.add(LeakyReLU(alpha=0.01))
      discriminator.add(Dropout(0.5))
      
      discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
      discriminator.add(BatchNormalization())
      discriminator.add(LeakyReLU(alpha=0.01))
      discriminator.add(Dropout(0.5))
      
      discriminator.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
      discriminator.add(BatchNormalization())
      discriminator.add(LeakyReLU(alpha=0.01))
      discriminator.add(Dropout(0.5))
      
      discriminator.add(Conv2D(512, kernel_size=3, strides=2, padding='same'))
      discriminator.add(BatchNormalization())
      discriminator.add(LeakyReLU(alpha=0.01))
      discriminator.add(Dropout(0.5))
      
      discriminator.add(Conv2D(1024, kernel_size=3, strides=2, padding='same'))
      discriminator.add(BatchNormalization())
      discriminator.add(LeakyReLU(alpha=0.01))
      discriminator.add(Dropout(0.5))
      
      discriminator.add(Flatten())
      discriminator.add(Dense(1))
      discriminator.add(Activation('sigmoid'))
      return discriminator

def build_generator():
      generator = Sequential()
      generator.add(Dense(8*8*1024, input_shape=noise_shape))
      generator.add(Reshape((8,8,1024)))
      generator.add(BatchNormalization())
      generator.add(LeakyReLU(alpha=0.01))
      
      generator.add(Conv2DTranspose(512, kernel_size=3, strides=2, padding='same'))
      generator.add(BatchNormalization())
      generator.add(LeakyReLU(alpha=0.01))
      
      generator.add(Conv2DTranspose(256, kernel_size=3, strides=2, padding='same'))
      generator.add(BatchNormalization())
      generator.add(LeakyReLU(alpha=0.01))
      
      generator.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
      generator.add(BatchNormalization())
      generator.add(LeakyReLU(alpha=0.01))
      
      generator.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))
      generator.add(BatchNormalization())
      generator.add(LeakyReLU(alpha=0.01))
      
      generator.add(Conv2DTranspose(1,kernel_size=3, strides=1, padding='same'))
      generator.add(Activation('tanh'))
      return generator

discriminator = build_discriminator()
generator = build_generator()

# Create optimizers
d_optimizer = Adam(lr=0.0005, beta_1=0.3)
g_optimizer = Adam(lr=0.0005, beta_1=0.3)

# Compile and setup discriminator
discriminator.compile(d_optimizer, loss='binary_crossentropy')
discriminator.trainable = False
#true doesnt give good results

noise_input = Input(shape=noise_shape)
generated_image = generator(noise_input)
validity = discriminator(generated_image)

# Build and compile a combined model
combined = Model(noise_input, validity)
combined.compile(optimizer=g_optimizer, loss='binary_crossentropy')

epochs = 10
batch_size=32
half_batch = batch_size//2

if not os.path.isdir('logs/facev2'):
      os.makedirs('logs/facev2')

# Init TensorBoard callback
date = datetime.today().strftime('%m-%d_%H%M')
callback = TensorBoard(os.path.join('logs/facev2',date))
callback.set_model(combined)
train_names = ['g_loss', 'd_loss']

# Make checkpoint directory
if not os.path.isdir('checkpoints/facev2/%s' % date):
      os.makedirs('checkpoints/facev2/%s' % date)

# Make image directory
if not os.path.isdir('generated_images/facev2/%s' % date):
      os.makedirs('generated_images/facev2/%s' % date)

write_image_period = 1 # Write images every 20 epochs
num_images_to_write = 10 # Generate 10 images

# adding random loss
best_g_loss = math.inf

DATASET_PATH = r'sample'
all_images = os.listdir(DATASET_PATH)
num_images = len([img for img in all_images if img.endswith('.png')])

def get_batch(batch_size):
      image_indexes = np.random.randint(0, len(all_images), batch_size)
      images = np.array([np.expand_dims(np.array(Image.open(os.path.join(DATASET_PATH,all_images[i]))),axis=2) for i in image_indexes])
      return rescale(images.astype(np.float32), -1, 1, data_min=0, data_max=255)

for epoch in range(epochs):
      losses = []
      for _ in tqdm(range(num_images // batch_size)):
            # Train discriminator
            
            # Get random valid images
            images = get_batch(half_batch)
            
            # Generate random invalid images
            noises = np.random.uniform(-1, 1, size=(half_batch, *noise_shape))
            generated_images = generator.predict(noises)
            
            # Train on each half batch
            # Invalid images' ground truth is zeros
            discriminator_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((half_batch,1)))
            # Valid images' ground truth is ones
            discriminator_loss_real = discriminator.train_on_batch(images, np.ones((half_batch,1))*0.9)
            discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
            
            # Train generator
            
            # Create noise to generate images
            noises = np.random.uniform(-1, 1, size=(batch_size, *noise_shape))
            
            # Train using the combined model, since the error relies on the discriminator
            generator_loss = combined.train_on_batch(noises, np.ones((batch_size, 1)))
            
            # Add to list of losses
            losses.append((generator_loss, discriminator_loss))
      # Calculate losses and accuracies
      avg_g_loss = np.average([loss[0] for loss in losses])
      avg_d_loss = np.average([loss[1] for loss in losses])
      
      noises = np.random.uniform(-1, 1, size=(half_batch, *noise_shape))
      generated_images = generator.predict(noises)
      
      if epoch % write_image_period == 0:
            for i,img in enumerate(generated_images[:num_images_to_write]):
                  image = rescale(img.squeeze(), 0, 255, data_min=-1, data_max=1)
                  image = Image.fromarray(np.round(image).astype(np.uint8))
                  image.save('generated_images/facev2/%s/epoch%03d_%d.png'%(date,epoch,i))
      
      try:
            generator.save('checkpoints/facev2/{}/g_epoch{:04d}.h5'.format(date, epoch))
            discriminator.save('checkpoints/facev2/{}/d_epoch{:04d}.h5'.format(date, epoch))
            best_g_loss = avg_g_loss
      except:
            pass
      # Write log to TensorBoard
      write_log(callback, train_names, [avg_g_loss, avg_d_loss], epoch)
      # Print to console
      print('Epoch: {: 5d} [G loss: {: 10.6f}] [D loss: {: 10.6f}]'.format(epoch, avg_g_loss, avg_d_loss))

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




  0%|          | 0/1250 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
  0%|          | 1/1250 [00:10<3:42:07, 10.67s/it]/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
  0%|          | 0/1250 [00:00<?, ?it/s]

Epoch:     0 [G loss:   0.406078] [D loss:   0.356367]


  0%|          | 0/1250 [00:00<?, ?it/s]

...

Epoch:     7 [G loss:   0.153749] [D loss:   0.196809]


  0%|          | 0/1250 [00:00<?, ?it/s]

Epoch:     8 [G loss:   0.267871] [D loss:   0.234816]


100%|██████████| 1250/1250 [04:37<00:00,  4.51it/s]


Epoch:     9 [G loss:   0.312727] [D loss:   0.232052]


In [16]:
ls

celeba-dataset.zip  img_align_celeba.zip     list_landmarks_align_celeba.csv
checkpoints/        kaggle.json              logs/
dataset.npy         list_attr_celeba.csv     sample/
generated_images/   list_bbox_celeba.csv     sample_data/
img_align_celeba/   list_eval_partition.csv


In [0]:
!zip -r /content/generated_images.zip /content/generated_images

In [0]:
from google.colab import files
files.download('generated_images.zip')